<a href="https://colab.research.google.com/github/import-cj/SentimentAnalysis_Malayalam/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow
!pip install transformers
!pip install tensorflow_hub
!pip install tensorflow_text
!pip install bert-for-tf2 --quiet
!pip install sentencepiece
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11743068594634224191
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16118448128
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1789617263739469752
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
xla_global_id: 416903419
]


In [ ]:
import os
import numpy as np
import glob
import torch
import sys
import pandas as pd
import transformers
import tensorflow as tf
import gc
from scipy.spatial import distance
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from transformers import DistilBertTokenizer, DistilBertModel, BertTokenizer, BertModel, AutoTokenizer, AutoModel, AutoModelForMaskedLM
import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_datasets as tfds

In [ ]:
from bert import bert_tokenization
from bert import tokenization
import tensorflow_text as text
from torch import cuda
from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from tqdm import tqdm, trange

from pathlib import Path

from transformers import (
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
    RemBertModel,
    RemBertTokenizer,
)

device = "cuda" if torch.cuda.is_available() else "cpu"  
torch.device(device)  
print("Current Device :", device)

Current Device : cuda


In [ ]:
# packages
import pandas as pd
from collections import Counter
from sentence_transformers import SentenceTransformer
import numpy as np
import sklearn
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
class model:
  def __init__(self,df,model_name):
    self.tokenizer=None
    self.model=None
    self.tokenized_padded_text=None
    self.attention_mask=None
    self.textip=None
    self.pooledOp=None
    self.input_dfs=None
    self.data_frame=df
    self.feature_df=None
    self.model_name=None
    self.InitModel(model_name)
  
  def reset(self,df):
    self.tokenized_padded_text=None
    self.attention_mask=None
    self.textip=None
    self.pooledOp=None
    self.input_dfs=None
    self.data_frame=df
    self.feature_df=None
    torch.cuda.empty_cache()
    gc.collect()

  def InitModel(self,model_name="ai4bharat/indic-bert") :
    from transformers import DistilBertTokenizer, DistilBertModel, BertTokenizer, BertModel
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if model_name == 'google/rembert':
      self.tokenizer = RemBertTokenizer.from_pretrained(model_name, model_max_length=512)
      self.model = RemBertModel.from_pretrained(model_name).to(device)
    if model_name == 'xlm-roberta-base':
      self.model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)
      self.tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)
    elif model_name == 'bert-base-multilingual-cased':
      self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',  model_max_length=512)
      self.model = BertModel.from_pretrained("bert-base-multilingual-cased").to(device)
    elif model_name == 'distilbert-base-uncased':
      model_class, tokenizer_class, pretrained_weights = (DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')  
      self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights,  model_max_length=512)
      self.model = model_class.from_pretrained(pretrained_weights).to(device)
    else:
      self.model = AutoModel.from_pretrained(model_name).to(device)
      self.tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)
    self.model.eval()
    self.model_name=model_name
  
  def tokenize(self,column):
       torch.cuda.empty_cache()
       gc.collect()
       tokenized_text=column.apply((lambda x: self.tokenizer.encode(x, add_special_tokens=True, truncation=True)))
       max_len = 0
       k=0
       for i in tokenized_text.values:
        #if len(i) > 500:
        #  print(k,len(i), i)
        if len(i) > max_len:
         max_len = len(i)
        k+=1
       self.tokenized_padded_text = np.array([i + [0]*(max_len-len(i)) for i in tokenized_text.values])
       tokenized_text = None
       self.create_attention_mask()
       gc.collect()
       torch.cuda.empty_cache()

  def create_attention_mask(self):
      torch.cuda.empty_cache()
      gc.collect()
      self.attention_mask = np.where(self.tokenized_padded_text != 0, 1, 0)
      self.input_ids = torch.tensor(self.tokenized_padded_text).to(device)  
      self.attention_mask = torch.tensor(self.attention_mask).to(device)
      torch.cuda.empty_cache()
      gc.collect()

  def GetFeatures(self,input=None):
   torch.cuda.empty_cache()
   gc.collect()
   with torch.no_grad():
    last_hidden_states = self.model(self.input_ids, attention_mask=self.attention_mask)
   torch.cuda.empty_cache()
   self.features = last_hidden_states[0][:,0,:].cpu().numpy()
   self.features=pd.DataFrame(self.features)
   torch.cuda.empty_cache()
   gc.collect()
   return self.features

In [ ]:
class classifiers:

  def __init__(self,traindf,testdf, style):
    self.Createstaticsplit(traindf,testdf, style)

  def classify(self,svm=False,svm2=False,svm3=False,random_forest=False,xgboost=False,logistic_regression=False,maj=False)  :
      if svm==True:
        y=self.CreateSVMClassifier()
        self.y_svm1 = y
        #print('svm')
      if svm2==True:
        y=self.CreateSVMClassifier2()
        self.y_svm2 = y
        #print('svm2')
      if svm3==True:
        y=self.CreateSVMClassifier3()
        self.y_svm3 = y
        #print('svm3')
      if random_forest==True:
        y=self.RandomForestClassifier()
        self.y_rf = y
        #print('random_forest')
      if xgboost==True:
        y=self.XGBClassifier(3)
        self.y_xgboost = y
        #print('xgboost')
      if logistic_regression==True:
        y=self.LogisticRegression()
        self.y_lr = y
        #print('lr')
      if maj==True:
        y=self.MajorityVotingClassifier()
        self.y_lr = y
        #print('lr')
      return y

  def MajorityVotingClassifier(self):
        y_pred = list()
        for i in range(len(self.y_xgboost)):
            preds = list()
            preds.append(self.y_xgboost[i])
            preds.append(self.y_rf[i])
            preds.append(self.y_lr[i])
            preds.append(self.y_svm1[i])
            preds.append(self.y_svm2[i])
            preds.append(self.y_svm3[i])
            y_pred.append(max(set(preds), key=preds.count))
        return y_pred 

  def XGBClassifier(self,num_class=3):
     from xgboost import XGBClassifier
     classifier = XGBClassifier(n_estimators=500,learning_rate=1, max_depth=2,objective='multi:softmax',num_class=num_class)
     classifier.fit(self.train_features, self.train_labels)
     y_pred = classifier.predict(self.test_features)
     return y_pred

  def CreateSVMClassifier(self):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'poly',decision_function_shape='ovr', random_state = 0, C=1000)
    classifier.fit(self.train_features, self.train_labels)
  
    y_pred = classifier.predict(self.test_features)
    return y_pred

  def CreateSVMClassifier2(self):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf',decision_function_shape='ovr', random_state = 0, C=1000)
    classifier.fit(self.train_features, self.train_labels)
  
    y_pred = classifier.predict(self.test_features)
    return y_pred

  def CreateSVMClassifier3(self):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear',decision_function_shape='ovr', random_state = 0, C=10)
    classifier.fit(self.train_features, self.train_labels)
  
    y_pred = classifier.predict(self.test_features)
    return y_pred

  def RandomForestClassifier(self):
     from sklearn.ensemble import RandomForestClassifier
     classifier = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
     classifier.fit(self.train_features, self.train_labels)

     y_pred = classifier.predict(self.test_features)
     return y_pred

  def Createstaticsplit(self,traindf, testdf,style):
   if style == 1:
    self.train_features = traindf.iloc[:, :-1]
    self.test_features = testdf.iloc[:, :-1]
   else:
    count_vec = None
    if style==2:
      count_vec = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)
    else:
      count_vec = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)
    count_vec.fit(traindf['text'])
    self.train_features = count_vec.transform(traindf['text']) 
    self.test_features = count_vec.transform(testdf['text'])
   self.train_labels=traindf.sentiment
   self.test_labels=testdf.sentiment
  def annClassifier(self):
      import tensorflow as tf
      from sklearn.compose import ColumnTransformer
      from sklearn.preprocessing import OneHotEncoder
      ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
      feature_set = np.array(ct.fit_transform(self.features_set))
      train_features, test_features, train_labels, test_labels = train_test_split(feature_set, self.labels)

      ann = tf.keras.models.Sequential()
      ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
      ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
      ann.add(tf.keras.layers.Dense(units=1, activation='softmax'))
      ann.add(tf.keras.layers.Dense(units=1, activation='softmax'))
      ann.add(tf.keras.layers.Dense(units=1, activation='softmax'))
      ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
      ann.fit(train_features, train_labels, batch_size = 32, epochs = 200)

      y_pred = ann.predict(test_features)

      return accuracy_score(test_labels, y_pred),f1_score(test_labels, y_pred,average='macro')

  def LogisticRegression(self):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression 
    lr_clf = LogisticRegression(multi_class='multinomial', C= 5, max_iter=1001)
    lr_clf.fit(self.train_features, self.train_labels) 

    y_pred=lr_clf.predict(self.test_features)
    return y_pred

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"  
torch.device(device)  
print("Current Device :", device)

Current Device : cuda


In [ ]:
def ReadData():
 cols=["text","clean-text","sentiment"]
 nm = "youtube_comments_cleaned_dataset.csv"
 df = pd.read_csv(nm, encoding='utf-8', usecols=cols)
 nm = "newsdataset_cleaned.csv"
 ndf = pd.read_csv(nm, encoding='utf-8', usecols=cols)
 ndf = ndf[ndf['sentiment'] == "neutral"]
 return pd.concat([df,ndf],ignore_index=True)

def ExtractFeatures(dataset,labels,model_name):
  model_pipeline=model(dataset,model_name)
  #model_pipeline.tokenize(dataset.text)
  feature_df_mbert=model_pipeline.GetFeatures(dataset.text)
  feature_df_mbert['label']=labels
  return feature_df_mbert 

def SaveAsCsv(dataframe,modelname):
    filename='CD_'+str(modelname)+'.csv'
    dataframe.to_csv(filename,index=False)

def testtrainsplit(df):
  from sklearn.model_selection import train_test_split
  trainx = [] 
  testx = []
  for i in [0,1,2]:
     tdf = df[df['sentiment'] == i]
     t1, t2 = train_test_split(tdf, test_size=0.2, random_state=0)
     num=len(tdf)
     trainx.append(t1)
     testx.append(t2)
  return pd.concat(trainx),pd.concat(testx)

def run_experiment(style=None, model_name=None, stem=False, stopwords=False, subset=100):
  torch.cuda.empty_cache()
  gc.collect()
  dataset=ReadData()
  dataset['sentiment'] = dataset.sentiment.map({'neutral':0, 'negative':1, 'positive':2})
  mystopwords = ['ഇത്തരം', 'ഒരു', 'ഈ',  'ആണ്', 'വളരെ', 'ഞാൻ', 'തന്നെ', 'ആ', 'വീഡിയോ',  'എന്റെ', 'നമ്മുടെ', 'എല്ലാ', 'അത്', 'എന്ന', 'കൊണ്ട്', 'ഉള്ള', 'ഇത്',  'എല്ലാം', 'എനിക്ക്', 'ഓരോ',  'പിന്നെ',  'ഒക്കെ',  'നിങ്ങൾ', 'വേണ്ടി', 'മാത്രം', 'നിങ്ങളുടെ', 'നിന്നും', 'കൂടെ', 'ഏറ്റവും', 'ഇങ്ങനെ', 'അവരുടെ',  'കാരണം', 'ഇല്ല', 'കൂടുതൽ', '-', 'എല്ലാവർക്കും', 'അവർ', 'പോലും', 'വീണ്ടും', 'എന്നു', 'ചെയ്യാൻ', 'അവിടെ', 'സ്വന്തം', 'അതിന്റെ', 'ഇവിടെ', 'അങ്ങനെ',  'എല്ലാവരും',  'ഞങ്ങൾ',  'നമ്മൾ', 'നമുക്ക്',  'പക്ഷെ', 'നീ',  'പക്ഷേ', 'എന്നെ',  'എന്നാൽ',  'ആരും', 'അതിൽ',  'അല്ല', 'ശേഷം',  'എന്ത്', 'താങ്കൾ', 'എങ്ങനെ', 'അതാണ്',  'നിങ്ങൾക്ക്', ')', 'അവർക്ക്',  'ഏത്',  'മറ്റു', 'അവരെ',
'നല്ല', 'ഉണ്ട്','എന്ന്', 'പോലെ', 'ആയി', 'കണ്ടു', 'ഒന്നും', 'നിന്ന്', 'വരെ', 'ചെയ്തു', 'വന്നു', 'ആയിരുന്നു', 'അടുത്ത', 'ചെയ്യുന്ന', 'പല', 'വേറെ', 'ചില', 'മതി', 'ഒരിക്കലും', 'കുറിച്ച്', 'പറഞ്ഞ', 'വന്ന', 'താങ്കളുടെ', 'ചേട്ടൻ', 'ചേട്ടാ', 'ചെയ്ത', 'നൽകി', 'കണ്ട്', 'എൻറെ', 'വെച്ച്', 'യും', 'നിങ്ങളെ', 'പറഞ്ഞത്', 'മാത്രമേ', 'അവൻ', 'എന്തെങ്കിലും', 'എന്നത്', 'എന്താണ്', 'കഴിഞ്ഞു', 'അറിയാം', 'അതു', 'മാത്രമാണ്', 'അതിനെ', 'ചെയ്ത്', 'ഇവർ', 'സ്വയം', 'മുൻപ്', 'ഉള്ളത്', 'പേര്'
'ഇപ്പോൾ', 'ഇനി', 'ഇനിയും', 'ഇനി' , 'ഇപ്പോൾ' , 'എങ്കിൽ', 'അപ്പോൾ', 'അല്ലെങ്കിൽ', 'ഇതിൽ', 'ഇതൊക്കെ', 'ഇത്രയും', 'ഇതിന്റെ', 'ഞങ്ങളുടെ', 'ഇതാണ്' , 'പിന്നീട്']
  mystopwords = set(mystopwords)
  if stem:
    dataset['text'] = dataset['clean-text']
  if stopwords:
    def stopfilter(row):
      text = row['text']
      ntxt = filter(lambda x: x not in mystopwords, text.split())
      return " ".join(list(ntxt))
    dataset['text'] = dataset.apply(stopfilter, axis=1)
  if style == 1:
    name = model_name
    dflist = []
    model_pipeline=model(None, model_name)
    for k,g in dataset.groupby(np.arange(len(dataset))//subset):
      model_pipeline.reset(g)
      model_pipeline.tokenize(g.text)
      dflist.append(model_pipeline.GetFeatures(g.text))
    dataframe = pd.concat(dflist,ignore_index=True)
    dataframe['sentiment'] = dataset.sentiment
  elif style < 4:
    name = "Count vectorizer"
    if style==3:
      name = "Tfidf vectorizer"
    dataframe = dataset
  elif style == 4:
    name = "Sentence Transformer "+model_name
    smodel = SentenceTransformer(model_name)
    features = smodel.encode(list(dataset.text))
    dataframe = pd.DataFrame(features)
    dataframe['sentiment'] = dataset.sentiment
    style = 1
  else:
    return
  header="\nEXPERIMENT | feature-extract: "+name+" | stopwords : "+str(stopwords)+ " | stemmer:"+str(stem)
  print(header)
  underline="-"*(len(header))
  underline+= "--\n\n"
  print(underline)
  classify(dataframe, style)
  torch.cuda.empty_cache()
  gc.collect()

In [ ]:
def classify(df, style):
      t1,t2=testtrainsplit(df)
      classifier_class=classifiers(t1,t2,style)
      test_labels=t2.sentiment
   
      def doClassifier(name, **kwargs):
        nonlocal test_labels
        y_pred=classifier_class.classify(**kwargs)
        print("\t\t\t"+name)
        print(classification_report(test_labels, y_pred, labels=[0,1,2]))
   
      doClassifier(name = "Logistic Regression", logistic_regression=True)
      #doClassifier(name = "SVM poly kernel",svm=True)
      doClassifier(name = "SVM rbf kernel",svm2=True)
      doClassifier(name = "SVM linear kernel",svm3=True)
      doClassifier(name = "Random Forest",random_forest=True)
      doClassifier(name = "XGBoost",xgboost=True)
      #doClassifier(name = "Majority Voting classifier of all classifiers", maj=True)
   

In [ ]:
#1/0#run_experiment(style=2, model_name=None, stem=True, stopwords=True, subset=100)

In [ ]:
#run_experiment(style=1, model_name='ai4bharat/indic-bert', stem=False, stopwords=False, subset=400)

In [ ]:
#run_experiment(style=4, model_name='aditeyabaral/sentencetransformer-indic-bert', stem=False, stopwords=False, subset=400)
ReadData()['sentiment'].value_counts()

positive    2394
negative    2391
neutral     1921
Name: sentiment, dtype: int64

In [ ]:
import copy

# load all experiments and download all models from huggingface
wordvec = ['ai4bharat/indic-bert',  'setu4993/LaBSE', 'google/muril-large-cased', 'ai4bharat/IndicBART', 'bert-base-multilingual-cased', 'google/rembert']
sentencevec = ['aditeyabaral/sentencetransformer-indic-bert', 'sentence-transformers/LaBSE']

experiments = []

for i in range(2,4):
  exp = {
    'style' : i,
    'model_name' : None,
    'stem' : False,
    'stopwords' : False,
    'subset' : None
  }
  experiments.append(exp)
  texp = copy.deepcopy(exp)
  texp['stem'] = True
  experiments.append(texp)
  texp = copy.deepcopy(exp)
  texp['stopwords'] = True
  experiments.append(texp)

for s in sentencevec:
  exp = {
    'style' : 4,
    'model_name' : s,
    'stem' : False,
    'stopwords' : False,
    'subset' : None
  }
  SentenceTransformer(s)
  experiments.append(exp)
  texp = copy.deepcopy(exp)
  texp['stem'] = True
  experiments.append(texp)
  texp = copy.deepcopy(exp)
  texp['stopwords'] = True
  experiments.append(texp)

for w in wordvec:
  exp = {
    'style' : 1,
    'model_name' : w,
    'stem' : False,
    'stopwords' : False,
    'subset' : 100
  }
  model(None, w)
  experiments.append(exp)
  texp = copy.deepcopy(exp)
  texp['stem'] = True
  experiments.append(texp)
  texp = copy.deepcopy(exp)
  texp['stopwords'] = True
  experiments.append(texp)

Downloading:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/823 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at google/muril-large-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at ai4bharat/IndicBART were not used when initializing MBartModel: ['final_logits_bias', 'lm_head.weight']
- This IS expected if you are initializing MBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# run all
for exp in experiments:
  run_experiment(**exp)



EXPERIMENT | feature-extract: Count vectorizer | stopwords : False | stemmer:False
-------------------------------------------------------------------------------------


			Logistic Regression
              precision    recall  f1-score   support

           0       0.65      0.65      0.65       385
           1       0.68      0.72      0.70       479
           2       0.77      0.72      0.75       479

    accuracy                           0.70      1343
   macro avg       0.70      0.70      0.70      1343
weighted avg       0.70      0.70      0.70      1343

			SVM rbf kernel
              precision    recall  f1-score   support

           0       0.59      0.66      0.62       385
           1       0.64      0.65      0.65       479
           2       0.77      0.67      0.72       479

    accuracy                           0.66      1343
   macro avg       0.67      0.66      0.66      1343
weighted avg       0.67      0.66      0.67      1343

			SVM linear kernel
    

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



EXPERIMENT | feature-extract: ai4bharat/indic-bert | stopwords : False | stemmer:False
-----------------------------------------------------------------------------------------


			Logistic Regression
              precision    recall  f1-score   support

           0       0.67      0.63      0.65       385
           1       0.59      0.72      0.65       479
           2       0.62      0.51      0.56       479

    accuracy                           0.62      1343
   macro avg       0.63      0.62      0.62      1343
weighted avg       0.62      0.62      0.62      1343

			SVM rbf kernel
              precision    recall  f1-score   support

           0       0.72      0.66      0.69       385
           1       0.62      0.70      0.66       479
           2       0.67      0.62      0.64       479

    accuracy                           0.66      1343
   macro avg       0.67      0.66      0.66      1343
weighted avg       0.66      0.66      0.66      1343

			SVM linear ker

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



EXPERIMENT | feature-extract: ai4bharat/indic-bert | stopwords : False | stemmer:True
----------------------------------------------------------------------------------------


			Logistic Regression
              precision    recall  f1-score   support

           0       0.68      0.63      0.65       385
           1       0.59      0.74      0.65       479
           2       0.66      0.53      0.58       479

    accuracy                           0.63      1343
   macro avg       0.64      0.63      0.63      1343
weighted avg       0.64      0.63      0.63      1343

			SVM rbf kernel
              precision    recall  f1-score   support

           0       0.72      0.64      0.68       385
           1       0.65      0.71      0.68       479
           2       0.66      0.66      0.66       479

    accuracy                           0.67      1343
   macro avg       0.68      0.67      0.67      1343
weighted avg       0.67      0.67      0.67      1343

			SVM linear kerne

...

			Logistic Regression
              precision    recall  f1-score   support

           0       0.73      0.70      0.72       385
           1       0.75      0.78      0.76       479
           2       0.78      0.78      0.78       479

    accuracy                           0.76      1343
   macro avg       0.75      0.75      0.75      1343
weighted avg       0.75      0.76      0.75      1343

			SVM rbf kernel
              precision    recall  f1-score   support

           0       0.68      0.66      0.67       385
           1       0.73      0.75      0.74       479
           2       0.74      0.73      0.74       479

    accuracy                           0.72      1343
   macro avg       0.72      0.72      0.72      1343
weighted avg       0.72      0.72      0.72      1343

			SVM linear kernel
              precision    recall  f1-score   support

           0       0.66      0.67      0.67       385
           1       0.73      0.74      0.73       479
           2 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


			Logistic Regression
              precision    recall  f1-score   support

           0       0.73      0.70      0.71       385
           1       0.70      0.75      0.73       479
           2       0.75      0.73      0.74       479

    accuracy                           0.73      1343
   macro avg       0.73      0.73      0.73      1343
weighted avg       0.73      0.73      0.73      1343

			SVM rbf kernel
              precision    recall  f1-score   support

           0       0.68      0.69      0.68       385
           1       0.70      0.75      0.72       479
           2       0.75      0.69      0.72       479

    accuracy                           0.71      1343
   macro avg       0.71      0.71      0.71      1343
weighted avg       0.71      0.71      0.71      1343

			SVM linear kernel
              precision    recall  f1-score   support

           0       0.69      0.71      0.70       385
           1       0.70      0.74      0.72       479
           2 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


			Logistic Regression
              precision    recall  f1-score   support

           0       0.72      0.68      0.70       385
           1       0.72      0.75      0.73       479
           2       0.74      0.74      0.74       479

    accuracy                           0.73      1343
   macro avg       0.73      0.72      0.72      1343
weighted avg       0.73      0.73      0.73      1343

			SVM rbf kernel
              precision    recall  f1-score   support

           0       0.68      0.66      0.67       385
           1       0.71      0.75      0.73       479
           2       0.75      0.72      0.74       479

    accuracy                           0.71      1343
   macro avg       0.71      0.71      0.71      1343
weighted avg       0.71      0.71      0.71      1343

			SVM linear kernel
              precision    recall  f1-score   support

           0       0.67      0.68      0.67       385
           1       0.71      0.70      0.70       479
           2 